<a href="https://colab.research.google.com/github/murilofarias10/Python/blob/main/build_ai_based_textworld_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚀 Build an AI-Powered Text World 🚀  

This Colab will guide you through creating an AI-powered Text World based on classic text adventures. The goal is to build it using one of the adventures from [textadventures.co.uk](https://textadventures.co.uk/).  

## Get Free Access to LLMs  

You can access powerful AI models, including **Llama 3.2**, with just **$1**, which gives you **over 1 million tokens** (equivalent to almost **300 book pages**). Simply **generate a token from [Together AI](https://docs.together.ai/docs/quickstart)** to get started.  

To use Hugging Face models, go to [Hugging Face](https://huggingface.co/) and in your profile settings, navigate to **Access Tokens** to generate your own token.  

## What to Do  
- **Try It Out**: Run all the code below to see how it works.  
- **Challenge**: Build an **AI-powered Text World** based on a game from **textadventures.co.uk**.  
- **Extra Challenges**:  
  - Generate a **graph** showing the locations and connections using **Graphviz**.  
  - Build a **GUI** for a more interactive experience!  
- **Deadline**: Submit your solution by **February 14**.  
- **Share Your Work**: Send your solution to **officialvancouverai@gmail.com** or to me directly on LinkedIn or WhatsApp (Issam Laradji)

Happy Coding! 🚀

In [ ]:
# Install the Libraries
!pip install -q together gradio==4.44.1

In [ ]:
# Get Free 1 dollar Together token at Together AI at https://api.together.xyz/settings/api-keys
import getpass
TOKEN = getpass.getpass()

··········


## 1. Setting Up Helper Functions

In [ ]:
# Add your utilities or helper functions to this file.
import os
import json
import gradio as gr
from together import Together
import random


def load_world(filename):
    with open(filename, "r") as f:
        return json.load(f)


def save_world(world, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w") as f:
        json.dump(world, f)


def get_together_api_key():
    together_api_key = TOKEN
    return together_api_key


def run_action(message, history, model):

    if message == "start game":
        return start
    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
Write only 3-5 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""

    world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character:  {character}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info},
    ]

    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.completions.create(model=model, messages=messages)

    result = model_output.choices[0].message.content
    return result


def main_loop(message, history, model="meta-llama/Llama-3-70b-chat-hf"):
    return run_action(message, history, model)


def get_game_state(inventory={}):
    world = load_world("build_ai_game/outputs/YourWorld_L2.json")
    # Filter kingdoms to only include those with towns that have key NPCs
    valid_kingdoms = {
        k: v
        for k, v in world["kingdoms"].items()
        if any("npcs" in town for town in v["towns"].values())
    }
    kingdom = valid_kingdoms[random.choice(list(valid_kingdoms.keys()))]
    # kingdom = world["kingdoms"][random.choice(list(world["kingdoms"].keys()))]
    # pick a random town
    town = kingdom["towns"][random.choice(list(kingdom["towns"].keys()))]
    # pick a random character
    character = town["npcs"][random.choice(list(town["npcs"].keys()))]

    # kingdom = world["kingdoms"]["Eldrida"]
    # town = kingdom["towns"]["Luminaria"]
    # character = town["npcs"]["Elwyn Stormbringer"]
    start = world.get("start", "")

    game_state = {
        "world": world["description"],
        "kingdom": kingdom["description"],
        "town": town["description"],
        "character": character["description"],
        "start": start,
        "inventory": inventory,
        "world_state": world,
    }
    return game_state


def run_action(message, history, game_state):

    if message == "start game":
        return game_state["start"]

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""

    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info},
    ]

    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    client = Together(api_key=get_together_api_key())
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf", messages=messages
    )

    result = model_output.choices[0].message.content
    return result


def start_game(main_loop, share=False):
    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=550, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(
            placeholder="What do you do next?", container=False, scale=7
        ),
        title="AI RPG",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Look around", "Continue the story"],
        cache_examples=False,
        retry_btn="Retry",
        undo_btn="Undo",
        clear_btn="Clear",
    )
    demo.launch(share=share, server_name="0.0.0.0")


everyone_content_policy = {
    "policy": """O1: Violence and Hate.
Should not
- Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics.
- Use slurs based on personal characteristics
- Write gruesome / graphic violence.
Can
- Write tame fantasy violence
O2: Sexual Content.
Should not
- Write any somewhat sexual content that mentions sexual functions or sexual body parts.
- Write any content that would be inappropriate for kids.
Can
- Write romantic content (including kissing, flirting etc...)
O3: Self-Harm.
Should not
- Encourage people to harm themselves.
- Romanticize or condone self-harm or suicide.
- Write story depicting suicide.
O4: Profanity.
Should not
- Write with any profane language that wouldn't be appropriate for kids.
Can
- Write with kid appropriate language
O5: Regulated or Controlled Substances.
Should not
- Write stories that depicts, glorifies or encourages drug use."""
}


def is_safe(message):
    # Build the prompt with embedded values
    prompt = f"""[INST] Task: Check if there is unsafe content in 'user' messages in conversations according to our safety policy with the below categories.

    <BEGIN UNSAFE CONTENT CATEGORIES>
    {everyone_content_policy['policy']}
    <END UNSAFE CONTENT CATEGORIES>

    <BEGIN CONVERSATION>
    user: {message}

    Provide your safety assessment for user in the above conversation:
    - First line must read 'safe' or 'unsafe'.
    - If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""

    client = Together(api_key=get_together_api_key())

    response = client.completions.create(
        model="Meta-Llama/LlamaGuard-2-8b",
        prompt=prompt,
    )
    result = response.choices[0].text
    return result.strip() == "safe"

import textwrap

def generate_pretty_text(world_data):
    def wrap(text):
        return '\n'.join(textwrap.wrap(text, width=50))

    text = f"""
🌍 WORLD: {world_data['name']}
{'=' * (len(world_data['name']) + 10)}

{wrap(world_data['description'])}

"""
    for kingdom_name, kingdom in world_data['kingdoms'].items():
        text += f"""
🏰 KINGDOM: {kingdom['name']}
{'=' * (len(kingdom['name']) + 12)}

{wrap(kingdom['description'])}

"""
        for town_name, town in kingdom['towns'].items():
            text += f"""
🏙️ TOWN: {town['name']}
{'=' * (len(town['name']) + 8)}

{wrap(town['description'])}

"""
            if 'npcs' in town:
                for npc_name, npc in town['npcs'].items():
                    text += f"""
🧑 NPC: {npc['name']}
{'=' * (len(npc['name']) + 7)}

{wrap(npc['description'])}

"""
    text += f"""

✨ START SCENE
=============

{wrap(world_data['start'])}
"""
    return text



### 2. Create a World Consisting of Kingdoms, Towns, and NPCs

In [ ]:
def get_system_prompt():
    """Returns the system prompt for world generation"""
    return """
    Your job is to help create interesting fantasy worlds that \
    players would love to play in.
    Instructions:
    - Only generate in plain text without formatting.
    - Use simple clear language without being flowery.
    - You must stay below 3-5 sentences for each description.
    """


def get_world_prompt():
    """Returns the prompt for generating the world"""
    return """
    Generate a creative description for a unique fantasy world with an
    interesting concept around cities build on the backs of massive beasts.

    Output content in the form:
    World Name: <WORLD NAME>
    World Description: <WORLD DESCRIPTION>

    World Name:"""


def get_kingdom_prompt(world):
    """Returns the prompt for generating kingdoms based on the world"""
    return f"""
    Create 3 different kingdoms for a fantasy world.
    For each kingdom generate a description based on the world it's in. \
    Describe important leaders, cultures, history of the kingdom.\

    Output content in the form:
    Kingdom 1 Name: <KINGDOM NAME>
    Kingdom 1 Description: <KINGDOM DESCRIPTION>
    Kingdom 2 Name: <KINGDOM NAME>
    Kingdom 2 Description: <KINGDOM DESCRIPTION>
    Kingdom 3 Name: <KINGDOM NAME>
    Kingdom 3 Description: <KINGDOM DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom 1"""


def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \

    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}

    Town 1 Name:"""


def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": get_system_prompt()},
            {"role": "user", "content": get_town_prompt(world, kingdom)},
        ],
    )
    towns_output = output.choices[0].message.content

    towns = {}
    for output in towns_output.split("\n\n"):
        town_name = output.strip().split("\n")[0].split("Name: ")[1].strip()
        print(f"- {town_name} created")

        town_description = (
            output.strip().split("\n")[1].split("Description: ")[1].strip()
        )

        town = {
            "name": town_name,
            "description": town_description,
            "world": world["name"],
            "kingdom": kingdom["name"],
        }
        towns[town_name] = town
    kingdom["towns"] = towns


def get_npc_prompt(world, kingdom, town):
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \

    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>

    World Name: {world['name']}
    World Description: {world['description']}

    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}

    Town Name: {town['name']}
    Town Description: {town['description']}

    Character 1 Name:"""


def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": get_system_prompt()},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)},
        ],
        temperature=0,
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split("\n\n"):
        npc_name = output.strip().split("\n")[0].split("Name: ")[1].strip()
        print(f'- "{npc_name}" created')

        npc_description = (
            output.strip().split("\n")[1].split("Description: ")[1].strip()
        )

        npc = {
            "name": npc_name,
            "description": npc_description,
            "world": world["name"],
            "kingdom": kingdom["name"],
            "town": town["name"],
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs


def generate_world(client):
    """Generates a new fantasy world using the AI model"""
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": get_system_prompt()},
            {"role": "user", "content": get_world_prompt()},
        ],
    )

    world_output = output.choices[0].message.content.strip()
    return {
        "name": world_output.split("\n")[0].strip().replace("World Name: ", ""),
        "description": "\n".join(world_output.split("\n")[1:])
        .replace("World Description:", "")
        .strip(),
    }


def generate_kingdoms(client, world):
    """Generates kingdoms for the given world"""
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": get_system_prompt()},
            {"role": "user", "content": get_kingdom_prompt(world)},
        ],
    )

    kingdoms = {}
    kingdoms_output = output.choices[0].message.content

    for output in kingdoms_output.split("\n\n"):
        kingdom_name = output.strip().split("\n")[0].split("Name: ")[1].strip()
        print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
        kingdom_description = (
            output.strip().split("\n")[1].split("Description: ")[1].strip()
        )
        kingdom = {
            "name": kingdom_name,
            "description": kingdom_description,
            "world": world["name"],
        }
        kingdoms[kingdom_name] = kingdom
    return kingdoms


if __name__ == "__main__":
    # Initialize Together API client
    client = Together(api_key=get_together_api_key())

    # Generate world
    world = generate_world(client)
    print(f"Generated world: {world['name']}")

    # Generate kingdoms
    kingdoms = generate_kingdoms(client, world)
    world["kingdoms"] = kingdoms

    # Generate towns for each kingdom
    for kingdom in kingdoms.values():
        create_towns(world, kingdom)

    # Generate NPCs for the first kingdom's towns
    first_kingdom = list(kingdoms.values())[0]
    for town in first_kingdom["towns"].values():
        create_npcs(world, first_kingdom, town)

    # Print sample outputs
    sample_town = list(first_kingdom["towns"].values())[0]
    sample_npc = list(sample_town["npcs"].values())[0]

    print(f"\nSample town description: {sample_town['description']}")
    print(f"\nSample NPC in {sample_town['name']}: {sample_npc['description']}")

    # Save the generated world
    save_world(world, "build_ai_game/outputs/YourWorld_L2.json")


Generated world: Kraelion
Created kingdom "Eldrida" in Kraelion
Created kingdom "Kordak" in Kraelion
Created kingdom "Valtoria" in Kraelion

Creating towns for kingdom: Eldrida...
- Brindlemark created
- Skypoint created
- Emberhaven created

Creating towns for kingdom: Kordak...
- Kragnir created
- Valtor's Peak created
- Ravenhollow created

Creating towns for kingdom: Valtoria...
- Wyrmhaven created
- Emberwatch created
- Luminaria created

Creating characters for the town of: Brindlemark...
- "Eira Shadowglow" created
- "Lyrien Starweaver" created
- "Kael Darkscale" created

Creating characters for the town of: Skypoint...
- "Kaelin Darkhaven" created
- "Lyrien Moonwhisper" created
- "Captain Thrain Blackwood" created

Creating characters for the town of: Emberhaven...
- "Kaida Blackwood" created
- "Eira Shadowglow" created
- "Arin the Wanderer" created

Sample town description: Located on the eastern flank of Arkeia, Brindlemark is a bustling trade hub where merchants and traveler

In [ ]:
# Visualize Cool stuff
desc_flag = False

if desc_flag:
  world = load_world("build_ai_game/outputs/YourWorld_L2.json")

  nicely_formatted_text = generate_pretty_text(world)
  print(nicely_formatted_text)

else:
  client = Together(api_key=get_together_api_key())

  response = client.images.generate(
      prompt="""
      Kraelion is a realm where ancient, gargantuan
creatures known as the "Colossi" roam the land,

      """, model="black-forest-labs/FLUX.1-schnell", steps=4
  )

  print(response.data[0].url)


https://api.together.ai/imgproxy/x9iX_t3TOWkMvBMFyLuczL_YpChFCb0M_6yMeyBlu_4/format:jpeg/aHR0cHM6Ly90b2dldGhlci1haS1iZmwtaW1hZ2VzLXByb2QuczMudXMtd2VzdC0yLmFtYXpvbmF3cy5jb20vaW1hZ2VzL2RlMjU4YjQxZGJiYmIwZTBhYTk0MmI1N2U4MzhjYmJhZWNlOGVkZTE0MzkyZTQyZmEyM2ZlMmY0ZjVmMDEyZjg_WC1BbXotQWxnb3JpdGhtPUFXUzQtSE1BQy1TSEEyNTYmWC1BbXotQ29udGVudC1TaGEyNTY9VU5TSUdORUQtUEFZTE9BRCZYLUFtei1DcmVkZW50aWFsPUFTSUFZV1pXNEhWQ0VSVlRHWTJEJTJGMjAyNTAyMDglMkZ1cy13ZXN0LTIlMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjUwMjA4VDAyMzExNFomWC1BbXotRXhwaXJlcz0zNjAwJlgtQW16LVNlY3VyaXR5LVRva2VuPUlRb0piM0pwWjJsdVgyVmpFR3NhQ1hWekxYZGxjM1F0TWlKR01FUUNJQTAxOEFCVHFQVjlmdHgxalZkVzFFTW1SWGw5VG85T3VhVlZ4VkRzU1dhTEFpQTdRVkpzakFQVG9Lb2QlMkJ0aVNzM1U4bVhWWUVYNzhWT1BnJTJCTndKZUklMkJpZ2lxWkJRaUUlMkYlMkYlMkYlMkYlMkYlMkYlMkYlMkYlMkYlMkY4QkVBQWFERFU1T0RjeU5qRTJNemM0TUNJTXYwUndlVHpPMGowMHFrWSUyQkt1MEVIMFBKWnRhY3lUMW44eE5Ed21IeFN6UiUyQlZnJTJGZlBFanpIV1JuZzAzMjltdjh1SmhrRWZ1cHpXZWElMkJ2Slc0NWxCVE5LR1BRam9SN05NcnglMkZkd04yTGptcDhxcFpCMUpiejBaa0Z

## 3. Create an Interactive World

In [ ]:
# Global demo variable for Gradio interface
demo = None


def start_game(main_loop, share=False):
    # Initializes and launches the Gradio chat interface
    global demo
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=500, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(
            placeholder="What do you do next?", container=False, scale=7
        ),
        title="AI RPG",
        theme="soft",
        examples=["Look around", "Continue the story"],
        cache_examples=False,
        retry_btn="Retry",
        undo_btn="Undo",
        clear_btn="Clear",
    )
    demo.launch(share=share, server_name="0.0.0.0")


def generate_game_start(client, world, kingdom, town, character):
    # Generates the initial game scenario using the AI model
    system_prompt = """You are an AI Game master. Your job is to create a
    start to an adventure based on the world, kingdom, town and character
    a player is playing as.
    Instructions:
    You must only use 2-4 sentences \
    Write in second person. For example: "You are Jack" \
    Write in present tense. For example "You stand at..." \
    First describe the character and their backstory. \
    Then describes where they start and what they see around them."""

    world_info = f"""
    World: {world}
    Kingdom: {kingdom}
    Town: {town}
    Your Character: {character}
    """

    model_output = client.chat.completions.create(
        model="nvidia/Llama-3.1-Nemotron-70B-Instruct-HF",
        temperature=1.0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": world_info + "\nYour Start:"},
        ],
    )

    return model_output.choices[0].message.content


def run_action(message, history, game_state):
    # Processes player actions and generates responses
    if message == "start game":
        return game_state["start"]

    system_prompt = """You are an AI Game master. Your job is to write what \
    happens next in a player's adventure game.\
    Instructions: \
    You must on only write 1-3 sentences in response. \
    Always write in second person present tense. \
    Ex. (You look north and see...)"""

    world_info = f"""
    World: {game_state['world']}
    Kingdom: {game_state['kingdom']}
    Town: {game_state['town']}
    Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info},
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf", messages=messages
    )

    return model_output.choices[0].message.content


if __name__ == "__main__":
    # Initialize the Together API client
    client = Together(api_key=get_together_api_key())

    # Load and initialize game world
    game_state = get_game_state()
    world = game_state["world"]
    kingdom = game_state["kingdom"]
    town = game_state["town"]
    character = game_state["character"]
    world_state = game_state["world_state"]

    # Generate the game start
    start = generate_game_start(client, world, kingdom, town, character)
    print(start)

    # Save the world state
    world_state["start"] = start
    save_world(world_state, "build_ai_game/outputs/YourWorld_L2.json")

    # Define main loop for the game
    def main_loop(message, history):
        return run_action(message, history, game_state)

    # Start the game
    start_game(main_loop, True)


**Your Character in the Moment:**
You are Lyrien, your silver hair whipping about your face as the winds of Skypoint buffet you. Your typically calm demeanor is slightly frayed, as you've just received a cryptic, anonymous note hinting at a breakthrough in your search for the forbidden text your mentor was last known to be researching. The note bears a single, enigmatic phrase: "Echoes of Eternity resonate within the Skyfest's symphony."

**Your Surroundings:**
You stand at the edge of Skypoint, overlooking the bustling town square where the final preparations for Skyfest are underway. Colorful, intricately designed flying machines of all shapes and sizes adorn the sky, their inventors making last-minute adjustments. Below, the town's inhabitants hurry about, setting up stalls and securing structures against the inevitable wind gusts that come with Arkeia's movements. To your left, the imposing silhouette of the Eldrid Library's Skypoint annex rises, its windows glowing with a soft, et

## 4. Create a Safe World

In [ ]:
def initialize_client():
    # Creates and returns a Together API client instance
    return Together(api_key=get_together_api_key())


def get_safety_policies():
    # Returns three different content policies: safe, moderate, and mature
    safe_content_policy = {
        "policy": """O1: Violence and Hate.
Should not
- Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics.
- Use slurs based on personal characteristics
- Write gruesome / graphic violence.
Can
- Write tame fantasy violence
O2: Sexual Content.
Should not
- Write any somewhat sexual content that mentions sexual functions or sexual body parts.
- Write any content that would be inappropriate for kids.
Can
- Write romantic content (including kissing, flirting etc...)
O3: Self-Harm.
Should not
- Encourage people to harm themselves.
- Romanticize or condone self-harm or suicide.
- Write story depicting suicide.
O4: Profanity.
Should not
- Write with any profane language that wouldn't be appropriate for kids.
Can
- Write with kid appropriate language
O5: Regulated or Controlled Substances.
Should not
- Write stories that depicts, glorifies or encourages drug use."""
    }

    moderate_content_policy = {
        "policy": """01: Violence and Hate
Should not
- Express racist, or offensive statements or slurs about people based on sensitive personal characteristics.
- Write graphics depicitions of gore that would make people very uncomfortable.
Can
- Write about violence or hate in a storytelling context.
O2: Sexual Content.
Should not
- Write sexually explicity content.
- Use any vulgar / sexual language
Can
- Write romantic content (including kissing, flirting etc...)
O3: Self-Harm.
Should not
- Encourage, or glorify self-harm or suicide."""
    }

    mature_content_policy = {
        "policy": """O2: Sexual Content.
Should not
- Write about graphic or illegal content.
Can
- Write about mature content between adults"""
    }

    return safe_content_policy, moderate_content_policy, mature_content_policy


def check_content_safety(client, text, policy):
    # Checks if given text complies with specified safety policy
    prompt = f"""[INST] Task: Check if there is unsafe content in 'user' messages in conversations according to our safety policy with the below categories.

    <BEGIN UNSAFE CONTENT CATEGORIES>
    {policy['policy']}
    <END UNSAFE CONTENT CATEGORIES>

    <BEGIN CONVERSATION>
    user: {text}

    Provide your safety assessment for user in the above conversation:
    - First line must read 'safe' or 'unsafe'.
    - If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""

    response = client.completions.create(
        model="Meta-Llama/LlamaGuard-2-8b",
        prompt=prompt,
    )
    return response.choices[0].text


def is_safe(client, message, policy):
    # Returns True if message is safe according to given policy, False otherwise
    prompt = f"""[INST] Task: Check if there is unsafe content in
    'user' messages in conversations according to our safety policy
    with the below categories.

    <BEGIN UNSAFE CONTENT CATEGORIES>
    {policy['policy']}
    <END UNSAFE CONTENT CATEGORIES>

    <BEGIN CONVERSATION>
    user: {message}

    Provide your safety assessment for user in the above conversation:
    - First line must read 'safe' or 'unsafe'.
    - If unsafe, a second line must include a comma-separated list of
    violated categories. [/INST]"""

    response = client.completions.create(
        model="Meta-Llama/LlamaGuard-2-8b",
        prompt=prompt,
    )

    result = response.choices[0].text
    return result.strip() == "safe"


def main_loop(client, message, history, safe_policy):
    # Main game loop that processes player actions and ensures safe content
    if not is_safe(client, message, safe_policy):
        return "This is foul language please behave"

    game_state = get_game_state()
    result = run_action(message, history, game_state)

    if is_safe(client, result, safe_policy):
        return result
    else:
        return "Invalid output."


def perform_initial_safety_check(client, character):
    # Performs initial safety check on character description
    safety_check = client.chat.completions.create(
        model="Meta-Llama/LlamaGuard-2-8b",
        messages=[{"role": "user", "content": character}],
    )
    return safety_check.choices[0].message.content


def test_safety_policies(client, policies):
    # Tests safety checks with different policy levels
    safe_policy, moderate_policy, mature_policy = policies
    check_text = "Kill the troll."

    print("Testing safety checks with different policies:")
    print("\nSafe policy check:")
    print(check_content_safety(client, check_text, safe_policy))
    print("\nModerate policy check:")
    print(check_content_safety(client, check_text, moderate_policy))
    print("\nMature policy check:")
    print(check_content_safety(client, check_text, mature_policy))


def create_game_loop(client, safe_policy):
    # Creates and returns the game loop function with necessary dependencies
    def game_loop(message, history):
        return main_loop(client, message, history, safe_policy)

    return game_loop


if __name__ == "__main__":
    # Initialize the client and get game state
    client = initialize_client()
    game_state = get_game_state()
    character = game_state["character"]

    # Display character information
    print("Character Description:", character)

    # Perform and display initial safety check
    safety_result = perform_initial_safety_check(client, character)
    print(f"prompt safety check: {safety_result}")
    print(" ")

    # Get and test safety policies
    policies = get_safety_policies()
    test_safety_policies(client, policies)

    # Create game loop and start the game
    game_loop = create_game_loop(client, policies[0])  # Using safe_policy
    start_game(game_loop, True)


Character Description: Kaida is a 25-year-old pyromancer with short, spiky black hair and piercing emerald eyes. She wears a black leather apron over her flame-patterned robes, symbolizing her apprenticeship to the esteemed Firemaster Thorne in Emberhaven. Kaida's passion for mastering the Eternal Flame is driven by her desire to prove herself worthy of her late father's legacy, a renowned pyromancer who perished in a tragic accident. She struggles with feelings of inadequacy and fears that her own magic may one day consume her.
prompt safety check: safe
 
Testing safety checks with different policies:

Safe policy check:
unsafe
O1

Moderate policy check:
unsafe
O1

Mature policy check:
safe
Closing server running on port: 7861
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5c5f74b012657e61ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Termin

## 5. Create a Stateful World (You have a persistent Inventory)

In [ ]:
import json
from together import Together


def initialize_game_client():
    # Initialize the Together AI client with API key
    return Together(api_key=get_together_api_key())


def detect_inventory_changes(game_state, output):
    # System prompt for inventory management
    system_prompt = """You are an AI Game Assistant. \
    Your job is to detect changes to a player's \
    inventory based on the most recent story and game state.
    If a player picks up, or gains an item add it to the inventory \
    with a positive change_amount.
    If a player loses an item remove it from their inventory \
    with a negative change_amount.
    Given a player name, inventory and story, return a list of json update
    of the player's inventory in the following form.
    Only take items that it's clear the player (you) lost.
    Only give items that it's clear the player gained.
    Don't make any other item updates.
    If no items were changed return {"itemUpdates": []}
    and nothing else.

    Response must be in Valid JSON
    Don't add items that were already added in the inventory

    Inventory Updates:
    {
        "itemUpdates": [
            {"name": <ITEM NAME>,
            "change_amount": <CHANGE AMOUNT>}...
        ]
    }
    """

    # Analyzes story output to detect changes in player's inventory
    client = initialize_game_client()
    inventory = game_state["inventory"]
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Current Inventory: {str(inventory)}"},
        {"role": "user", "content": f"Recent Story: {output}"},
        {"role": "user", "content": "Inventory Updates"},
    ]
    chat_completion = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        temperature=0.0,
        messages=messages,
    )
    response = chat_completion.choices[0].message.content
    result = json.loads(response)
    return result["itemUpdates"]


def update_inventory(inventory, item_updates):
    # Updates inventory based on detected changes and returns update message
    update_msg = ""

    for update in item_updates:
        name = update["name"]
        change_amount = update["change_amount"]

        if change_amount > 0:
            if name not in inventory:
                inventory[name] = change_amount
            else:
                inventory[name] += change_amount
            update_msg += f"\nInventory: {name} +{change_amount}"
        elif name in inventory and change_amount < 0:
            inventory[name] += change_amount
            update_msg += f"\nInventory: {name} {change_amount}"

        if name in inventory and inventory[name] < 0:
            del inventory[name]

    return update_msg


def run_action(message, history, game_state):
    # Processes player actions and generates story responses
    if message == "start game":
        return game_state["start"]

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...) \
Don't let the player use items they don't have in their inventory.
"""

    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}
Inventory: {json.dumps(game_state['inventory'])}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info},
    ]

    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    client = initialize_game_client()
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf", messages=messages
    )

    return model_output.choices[0].message.content


def main_loop(message, history):
    # Main game loop that processes input and manages game state
    output = run_action(message, history, game_state)

    safe = is_safe(output)
    if not safe:
        return "Not Safe - Please Behave"

    item_updates = detect_inventory_changes(game_state, output)
    update_msg = update_inventory(game_state["inventory"], item_updates)
    output += update_msg

    return output


if __name__ == "__main__":


    # Initialize game state with starting inventory
    game_state = get_game_state(
        inventory={
            "cloth pants": 1,
            "cloth shirt": 1,
            "goggles": 1,
            "leather bound journal": 1,
            "gold": 5,
        }
    )

    # Start the game
    start_game(main_loop, True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://79ba09c001708d462d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


https://api.together.ai/imgproxy/IspRxNVvGfKoJ5DNG2zQMIe52SvXdoZsL81cx7o4dGs/format:jpeg/aHR0cHM6Ly90b2dldGhlci1haS1iZmwtaW1hZ2VzLXByb2QuczMudXMtd2VzdC0yLmFtYXpvbmF3cy5jb20vaW1hZ2VzLzVkYjRmMzk1MzJiZWIwMmI2NmE2ZmY2ZmM5NjJlMTU4YWI2MTVjM2NlZGRmMTA4NjU5NDNmZDI0NDczODliNTQ_WC1BbXotQWxnb3JpdGhtPUFXUzQtSE1BQy1TSEEyNTYmWC1BbXotQ29udGVudC1TaGEyNTY9VU5TSUdORUQtUEFZTE9BRCZYLUFtei1DcmVkZW50aWFsPUFTSUFZV1pXNEhWQ05QSkhLVTM2JTJGMjAyNTAyMDglMkZ1cy13ZXN0LTIlMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjUwMjA4VDAyMjMxOFomWC1BbXotRXhwaXJlcz0zNjAwJlgtQW16LVNlY3VyaXR5LVRva2VuPUlRb0piM0pwWjJsdVgyVmpFR3NhQ1hWekxYZGxjM1F0TWlKR01FUUNJSHRiT1RHVlAxUkt1MVU4VVJJZ0d3JTJGWmNhdWkwQUpoUUljdVhKY1IzWkJjQWlCNnA3V3FNc3QyZFRRRW03VE5NMXdKRHc3cDdHSjZ2Y2hHaG5HMzViVUlQeXFaQlFpRSUyRiUyRiUyRiUyRiUyRiUyRiUyRiUyRiUyRiUyRjhCRUFBYUREVTVPRGN5TmpFMk16YzRNQ0lNOU5hSlBDSUJ1aVZRd1JqREt1MEVkVDN6cHdoUm5UTWxTdGVIWk9KV3E4TGNKU3YyZk9mMVpmd28lMkY1Y0trcnU3WjFOMmM3V1dLRUNxdm1BSTNPaTkyZ1NvVlgxak14bXBTU00lMkJ2ZlVEZHZWTmVpZXo5MDNkTU9MNEpsc0FBSlJOOFN